# Data Wrangling

In [169]:
# all modules

import warnings
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
from functools import reduce

import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import rc


In [2]:
# global stuff

warnings.filterwarnings('ignore')
%matplotlib inline
pd.set_option('display.max_rows', 13)
pd.set_option('display.max_columns', 13)

In [3]:
def get_dictEuropeanCountries():
    wiki_url= 'https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2'
    url= 'https://www.euro.who.int/en/countries'
    
    html_wikipage = requests.get(wiki_url).content
    html_eurocountries = requests.get(url).content
        
    countries_list = pd.read_html(html_wikipage, header=0)[2]
    countries_dict = dict(zip(countries_list['Code'], countries_list['Country name (using title case)']))

    soup = BeautifulSoup(html_eurocountries, 'lxml')
    table = soup.find_all('section', {'class':'clearfix'})

    all_contries = [content.text for content in table]
    eu_countries = list(filter(None, all_contries[0].split('\n')))

    european_countries_values = [val for k,val in countries_dict.items() for eu_c in eu_countries if val == eu_c]
    european_countries_key = [k for k,val in countries_dict.items() for eu_c in eu_countries if val == eu_c]

    return dict(zip(european_countries_key, european_countries_values))

## OPENING CLEANED TABLES

In [4]:
df_personal_info = pd.read_csv('../data/processed/personal_info.csv', index_col=[0])
df_career_info = pd.read_csv('../data/processed/career_info.csv', index_col=[0])
df_country_info = pd.read_csv('../data/processed/country_info.csv', index_col=[0])

In [5]:
df_career_info.head()

,uuid,dem_full_time_job,normalized_job_code,High_Ed,Low_Ed,Medium_Ed,No_Ed,Unknown_Ed,normalized_job_names
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,False,NaN,False,False,False,True,False,NaN
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,True,861a9b9151e11362eb3c77ca914172d0,True,False,False,False,False,Automatic Data Processing Planner
2,83127080-da3d-0133-c74f-0a81e8b09a82,False,NaN,False,False,False,False,True,NaN
3,15626d40-db13-0133-ea5c-0a81e8b09a82,True,049a3f3a2b5f85cb2971ba77ad66e10c,True,False,False,False,False,Data Coordinator
4,24954a70-db98-0133-4a64-0a81e8b09a82,True,f4b2fb1aa40f661488e2782b6d57ad2f,True,False,False,False,False,Database Developer


# TABLE 1 [country-job title-gender-quantity-percetnage]

In [6]:
########## FUNCTIONS TO EVAL ARGUMENT AT THE BEGINNING

def country_argparse_eval(country_argument, list_to_search):
    """
    Evaluates whether argument exists in df_country_info[country_names]
    """
    print(f'\t ··· Validating country argparse')
    
    if country_argument in list_to_search:
        print(f'\t\t >> country_argument found in ddbb')
        def_country_argument = country_argument
        return def_country_argument
    
    else:
        print(f'\t\t >> country_argument not found')
        def_country_argument = ""
        return def_country_argument

    
def country_argparse_filter(country_argument, analysis_df):
    """
    Return analysis df by argument input
    """
    list_to_search = analysis_df['country_names'].unique().tolist()
    c_arg = country_argparse_eval(country_argument, list_to_search)
    
    if c_arg == "":
        return analysis_df
    else:
        return analysis_df[analysis_df['country_names'] == c_arg]

    
def get_base_analysis_df(country_argument):
    """
    Returns base_df for analysis after evaluating country argument
    """
    dfs = [
        df_country_info[['uuid', 'country_names']],
        df_personal_info[['uuid', 'gender']],
        df_career_info[['uuid', 'normalized_job_names', 'dem_full_time_job', 'High_Ed', 'Low_Ed', 'Medium_Ed', 'No_Ed']], 
        ]

    df_final = reduce(lambda left,right: pd.merge(left,right,on='uuid'), dfs)
    
    return country_argparse_filter(country_argument.capitalize(), df_final)

In [7]:
def get_percentages_gender_by_job(base_analysis_df):
    
    # Variables.
    filtr = ['country_names','normalized_job_names','gender']
    
    drop_cols = ['uuid', 'dem_full_time_job', 
                 'High_Ed', 'Low_Ed', 'Medium_Ed', 'No_Ed', 
                 'totals_per_country']
    
    new_cols = ['quantity', 'percentage']

    # Add first col = quality
    df_job_gender = base_analysis_df.assign(quantity = 1)\
                         .drop(columns=drop_cols[0:-1])\
                         .groupby(filtr)\
                         .agg('count')\
                         .reset_index()
    
    # Generate totals_per_country
    df_total_per_country = df_job_gender.groupby(filtr[0])\
                        [filtr[1]]\
                        .nunique()\
                        .to_frame()\
                        .rename(columns={filtr[1]: drop_cols[-1]})
    
    df_job_gender = df_job_gender.merge(df_total_per_country, on=filtr[0])

    # Add second col == percentage and deleting totals_per_country when not need
    df_job_gender[new_cols[1]] = round(df_job_gender[new_cols[0]] / df_job_gender[drop_cols[-1]] * 100, 3)
    df_job_gender.drop(columns=[drop_cols[-1]], inplace=True)
    
    return df_job_gender
    

In [8]:
df_job_gender = get_percentages_gender_by_job(
                get_base_analysis_df(country_argument='SpAiN'))

	 ··· Validating country argparse
		 >> country_argument found in ddbb


In [9]:
df_job_gender

,country_names,normalized_job_names,gender,quantity,percentage
0,Spain,Analytical Data Miner,M,1,0.781
1,Spain,Automatic Data Processing Customer Liaison (AD...,F,5,3.906
2,Spain,Automatic Data Processing Customer Liaison (AD...,M,6,4.688
3,Spain,Automatic Data Processing Planner,F,5,3.906
4,Spain,Automatic Data Processing Planner,M,4,3.125
...,...,...,...,...,...
203,Spain,Survey Data Technician,F,1,0.781
204,Spain,Voice and Data Technician,F,1,0.781
205,Spain,Voice and Data Technician,M,1,0.781
206,Spain,Weight in Motion Field Data Collection Technician,F,2,1.562


# TABLE 3 [top 10 skills by education level]

In [10]:
df_analysis = get_base_analysis_df(country_argument='SpAiN')

	 ··· Validating country argparse
		 >> country_argument found in ddbb


In [11]:
df_high_ed = df_analysis[df_analysis['High_Ed']].drop(columns=['High_Ed', 'Low_Ed', 'Medium_Ed', 'No_Ed'])

In [12]:
df_high_ed[df_high_ed['gender'] == 'F'].head(3)
df_high_ed[df_high_ed['gender'] == 'M'].head(3)

,uuid,country_names,gender,normalized_job_names,dem_full_time_job
2212,72a48100-da38-0133-487f-0a81e8b09a82,Spain,M,Clinical Data Management Manager (CDM Manager),True
2218,6256a6a0-d961-0133-7e9a-0a81e8b09a82,Spain,M,NaN,False
2225,33a4bd30-d97d-0133-0b6d-0a81e8b09a82,Spain,M,Automatic Data Processing Planner,True


In [122]:
def top_skills_by_ed_level(base_analysis_df, number_skills, ed_level, gender_to_eval):
    
    
    # Variables
    filtr = 'normalized_job_names'
    drop_cols = ['uuid', 'country_names', 'gender', 'dem_full_time_job']
    
    # Filter by education level
    df_high_ed = base_analysis_df[df_analysis[ed_level]]\
                                .drop(columns=['High_Ed', 'Low_Ed', 'Medium_Ed', 'No_Ed'])
    # Getting only one gender
    if gender_to_eval == "All":
        df_high_ed
    else:        
        df_high_ed = df_high_ed[df_high_ed['gender'] == gender_to_eval]
    
    # Counting and sorting
    df_high_ed_skills_gender = df_high_ed.assign(counts = 1)\
                                     .drop(columns=drop_cols)\
                                     .groupby(filtr)\
                                     .agg('count')\
                                     .reset_index()

    df_high_ed_skills_gender.sort_values(by=['counts'], ascending=False, inplace=True)
    counts_by_gender = df_high_ed_skills_gender['counts'].tolist()
    
    pd = df_high_ed_skills_gender[filtr].reset_index().head(number_skills)
                                  
    pd = pd.rename(columns={filtr: ed_level}).drop(columns='index')
    counts_by_gender = counts_by_gender[:number_skills]
    
    # returns tuple 
    return pd, counts_by_gender

In [123]:
serie_top_skills_high_ed = top_skills_by_ed_level(base_analysis_df= get_base_analysis_df(
                                                                country_argument= 'Spain'), 
                                              number_skills= 5, 
                                              ed_level= 'Low_Ed',
                                              gender_to_eval= 'F')

	 ··· Validating country argparse
		 >> country_argument found in ddbb


In [124]:
serie_top_skills_high_ed[1]

[2, 1, 1, 1, 1]

In [211]:
def get_df_top_skills(country_argument, num_top_skills, list_dfs_cleaned, gender="All"):
    # Variables
    ed_levels = ['High_Ed', 'Medium_Ed', 'Low_Ed']
    path_to_save = 'data/results'

    # Tuples of top skills jobs and counters
    serie_top_skills_high_ed, counts_top_skills_high_ed = top_skills_by_ed_level(base_analysis_df=get_base_analysis_df(country_argument=country_argument, list_of_clean_df= list_dfs_cleaned),
                                                                                number_skills=num_top_skills,
                                                                                ed_level=ed_levels[0],
                                                                                gender_to_eval= gender)

    serie_top_skills_medium_ed, counts_top_skills_medium_ed = top_skills_by_ed_level(base_analysis_df=get_base_analysis_df(country_argument=country_argument, list_of_clean_df= list_dfs_cleaned),
                                                                                number_skills=num_top_skills,
                                                                                ed_level=ed_levels[1],
                                                                                gender_to_eval=gender)

    serie_top_skills_low_ed, counts_top_skills_low_ed = top_skills_by_ed_level(base_analysis_df=get_base_analysis_df(country_argument=country_argument, list_of_clean_df= list_dfs_cleaned),
                                                                                number_skills=num_top_skills,
                                                                                ed_level=ed_levels[2],
                                                                                gender_to_eval=gender)

    cols = dict(zip([n for n in range(5)], ['#' + str(n) for n in range(5)]))
    # Construccion of DF of jobs
    all_dfs = [serie_top_skills_high_ed, serie_top_skills_medium_ed, serie_top_skills_low_ed]
    result_dfs = pd.concat(all_dfs, axis=1, sort=False)

    df_to_save = result_dfs.T.rename(columns=cols)

    # Construction of DF of counts
    all_counts = [counts_top_skills_high_ed, counts_top_skills_medium_ed, counts_top_skills_low_ed]
    result_counts = pd.DataFrame(all_counts)

    # Save DataFrame to csv in data/results
    m_ac.save_df_to_csv(df_to_save,
                        path=path_to_save,  # Function adds hierarchy of files
                        name=f'df_top_skills')  # Name of csv
    return df_to_save, result_counts, gender

In [212]:
df_top_skills_M = get_df_top_skills(country_argument = 'Spain', 
                                  num_top_skills = 5)

TypeError: get_df_top_skills() missing 1 required positional argument: 'list_dfs_cleaned'

In [207]:
df_top_skills[0].shape

(3, 5)

In [156]:
counts_3

[8, 5, 2]

In [208]:
def get_stacked_bar_plot(tuple_top_skills):

    # Variables
    gender = tuple_top_skills[2]
    file_name = f'visualizacion_{gender}_distribution_top_skills'
    N = tuple_top_skills[0].shape[1] # shape is [5x3]
    ind = np.arange(N)    # the x locations for the groups
    width = 0.35       # the width of the bars: can also be len(x) sequence

    # list of values
    counts_1 = list(tuple_top_skills[1].iloc[:, 0])
    counts_2 = list(tuple_top_skills[1].iloc[:, 1])
    counts_3 = list(tuple_top_skills[1].iloc[:, 2])
    counts_4 = list(tuple_top_skills[1].iloc[:, 3])
    counts_5 = list(tuple_top_skills[1].iloc[:, 4])

    # stack_bars
    top_1 = plt.bar(ind, counts_1, width, bottom= 0, color='darkcyan')
    top_2 = plt.bar(ind, counts_2, width, bottom= counts_1, color='cadetblue')
    top_3 = plt.bar(ind, counts_3, width, bottom= [i+j for i,j in zip(counts_1, counts_2)], color='cyan')
    top_4 = plt.bar(ind, counts_4, width, bottom= [i+j+k for i,j,k in zip(counts_1, counts_2, counts_3)], color='lightseagreen')
    top_5 = plt.bar(ind, counts_5, width, bottom= [i+j+k+l for i,j,k,l in zip(counts_1, counts_2, counts_3, counts_4)], color='lightskyblue')

    plt.ylim(0, 45)
    # Legend for X and Y
    plt.title(f'VARIATION OF TOP SKILLS BY EDUCATION. GENDER = {gender}')
    plt.xticks(ind, ('high_ed', 'medium_ed', 'low_ed'))
    plt.legend((top_1[0], top_2[0], top_3[0], top_4[0], top_5[0]), ('#1', '#2', '#3', '#4', '#5'))

    #Save to data/results
    fig = plt.gcf()
    fig.set_size_inches(12, 20)
    fig.savefig(f'../data/results/{file_name}.png', dpi=100)

    return plt.show()


In [210]:
stacked_bar_plot(get_df_top_skills(country_argument = 'Spain', 
                                  num_top_skills = 5,
                                  gender = 'M'))

	 ··· Validating country argparse
		 >> country_argument found in ddbb
	 ··· Validating country argparse
		 >> country_argument found in ddbb
	 ··· Validating country argparse
		 >> country_argument found in ddbb


TypeError: stacked_bar_plot() missing 1 required positional argument: 'gender'

In [106]:
[-1*i for i in list(df_top_skills_F[1].iloc[0,:])]

[-5, -5, -4, -4, -4]

In [105]:
[-1*i for i in list(df_top_skills_F[1].iloc[1,:])]

[-4, -3, -3, -3, -3]

In [ ]:
    # list of values
    count_str = 'counts_'
    dict_names =  []
    for n in Number_of_stacks:
        dict_names.append
    counts_1 = list(tuple_top_skills[1].iloc[:, 0])
    counts_2 = list(tuple_top_skills[1].iloc[:, 1])
    counts_3 = list(tuple_top_skills[1].iloc[:, 2])
    counts_4 = list(tuple_top_skills[1].iloc[:, 3])
    counts_5 = list(tuple_top_skills[1].iloc[:, 4])


In [220]:
dict_names =  []
dict_lists =  []

for n in range(5):
    dict_names.append(f'count_{n}')
    dict_lists.append(list(df_top_skills_F[1].iloc[:, n]))
                      
dict_counts = dict(zip(dict_names, dict_lists)) 

In [221]:
dict_counts

{'count_0': [5, 4, 2],
 'count_1': [5, 3, 1],
 'count_2': [4, 3, 1],
 'count_3': [4, 3, 1],
 'count_4': [4, 3, 1]}

In [222]:
dict_counts['count_0']

[5, 4, 2]